In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! cp drive/MyDrive/IR_Project/cwishareddataset/traindevset/english/News_Train.tsv .
! cp drive/MyDrive/IR_Project/cwishareddataset/testset/english/News_Test.tsv .
! cp -r drive/MyDrive/IR_Project/cwishareddataset/traindevset/ .
! cp -r drive/MyDrive/IR_Project/cwishareddataset/testset/ .

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
word_label_dict = {}
seen_dict = {}

In [ ]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
def process_input(text):
  text = text.lower()
  sen_tok_list = []
  wpt = WordPunctTokenizer()
  sen_tok = wpt.tokenize(text)
  sen_tok_list.append(sen_tok)
  final_word_list = []
  wnlem = WordNetLemmatizer()
  for i in range(len(sen_tok_list)):
    for j in range(len(sen_tok_list[i])):
      sen_tok_list[i][j] = wnlem.lemmatize(sen_tok_list[i][j])
  for i in range(len(sen_tok_list)):
    temp_word_list = []
    for j in range(len(sen_tok_list[i])):
      if sen_tok_list[i][j].isalpha() == True and sen_tok_list[i][j].isnumeric() == False:
        temp_word_list.append(sen_tok_list[i][j])
    final_word_list.append(temp_word_list)
  return final_word_list

In [ ]:
import pandas as pd
import numpy as np
import string
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
def generate_final_word_list(file_path):
  dataframe = pd.read_csv(file_path,sep = '\t',header = None,names = ["ID","Sentence","complex_phrase_start_index","complex_phrase_end_index","Complex_phrase","num_native","num_n_native","native_complex","non-native complex","Binary values","probability values"])
  dataframe["Sentence"] = dataframe["Sentence"].apply(lambda x : x.lower())
  dataframe["Complex_phrase"] = dataframe["Complex_phrase"].apply(lambda x : x.lower())
  dataframe["Num_words"] = dataframe["Complex_phrase"].apply(lambda x : len(x.split()))
  dataframe = dataframe[dataframe["Num_words"] == 1]
  sentences_list = np.array(dataframe["Sentence"])
  labels = np.array(dataframe["Binary values"])
  complex_words = np.array(dataframe["Complex_phrase"])
  complex_word_dict = {complex_words[i]: labels[i] for i in range(len(labels))}
  sen_tok_list = []
  wpt = WordPunctTokenizer()
  for sentence in sentences_list:
    sen_tok = wpt.tokenize(sentence)
    sen_tok_list.append(sen_tok)
  wnlem = WordNetLemmatizer()
  for i in range(len(sen_tok_list)):
    for j in range(len(sen_tok_list[i])):
      sen_tok_list[i][j] = wnlem.lemmatize(sen_tok_list[i][j])
  final_word_list = []
  for i in range(len(sen_tok_list)):
    temp_word_list = []
    for j in range(len(sen_tok_list[i])):
      if sen_tok_list[i][j].isalpha() == True and sen_tok_list[i][j].isnumeric() == False:
        temp_word_list.append(sen_tok_list[i][j])
    final_word_list.append(temp_word_list)
  for sentence in final_word_list:
    for word in sentence:
      if word not in seen_dict:
        seen_dict[word] = 1
        if word not in complex_word_dict:
          word_label_dict[word] = 0
        else:
          word_label_dict[word] = complex_word_dict[word]

In [ ]:
len(word_label_dict)

0

In [ ]:
generate_final_word_list("News_Train.tsv")
generate_final_word_list("traindevset/english/WikiNews_Train.tsv")
generate_final_word_list("traindevset/english/Wikipedia_Train.tsv")

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
import string
wnlem = WordNetLemmatizer()
def complex_word_check(word):
  if(word in string.punctuation):
    return False
  word = wnlem.lemmatize(word)
  if word in word_label_dict:
    return word_label_dict[word]
  else:
    return False

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.3 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
model = TFBertForMaskedLM.from_pretrained('bert-large-cased')


All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-large-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [ ]:
!pip install wordfreq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 6.0 MB/s eta 0:00:00


In [ ]:
def get_top_k_predictions(input_string, k=5, tokenizer=tokenizer, model=model) -> str:

    tokenized_inputs = tokenizer(input_string, return_tensors="tf")
    outputs = model(tokenized_inputs["input_ids"])

    top_k_indices = tf.math.top_k(outputs.logits, k).indices[0].numpy()
    decoded_output = tokenizer.batch_decode(top_k_indices)
    mask_token = tokenizer.encode(tokenizer.mask_token)[1:-1]
    mask_index = np.where(tokenized_inputs['input_ids'].numpy()[0]==mask_token)[0][0]

    decoded_output_words = decoded_output[mask_index]

    return decoded_output_words   

In [ ]:
from wordfreq import zipf_frequency
def simplify_sentence(text):
  text = text.lower()
  wpt = WordPunctTokenizer()
  tokens = wpt.tokenize(text)
  complex_words = []
  index_of_complex_word = []
  for i in range(len(tokens)):
    if(complex_word_check(tokens[i]) == True):
      complex_words.append(tokens[i])
      index_of_complex_word.append(i)
  for j in range(len(complex_words)):
    encoded_text = tokenizer.encode(text, add_special_tokens=True)
    complex_word_index = encoded_text.index(tokenizer.convert_tokens_to_ids(complex_words[j]))
    encoded_text[complex_word_index] = tokenizer.mask_token_id
    text_with_mask = tokenizer.decode(encoded_text)
    predictedWords = get_top_k_predictions(input_string = text_with_mask)
    predictedWords = predictedWords.split()
    # print(predictedWords)
    max = -1000000
    word = ''
    for i in range(len(predictedWords)):
      temp = zipf_frequency(predictedWords[i], 'en') * (len(predictedWords) - i/2)
      if temp>max:
        max = temp
        word = predictedWords[i]
    tokens[index_of_complex_word[j]] = word
  sentence = ""
  for token in tokens:
    sentence += token + " "
  return sentence

In [ ]:
from nltk.tokenize import WordPunctTokenizer
text1 = 'Ebola is a fatal and deadly virus'
text2 = "Water is a vital resource."
text3 = 'The door to his residence was locked'
print("Original text is: {} \nSimplified text is: {}".format(text1,simplify_sentence(text1)))
print("Original text is: {} \nSimplified text is: {}".format(text3,simplify_sentence(text3)))
print("Original text is: {} \nSimplified text is: {}".format(text2,simplify_sentence(text2)))

Original text is: Ebola is a fatal and deadly virus 
Simplified text is: ebola is a rare and deadly disease! 
Original text is: The door to his residence was locked 
Simplified text is: the door to his room was locked 
Original text is: Water is a vital resource. 
Simplified text is: water is a natural resource . 


In [ ]:
# for input_text in list_texts:
#   new_text = input_text
#   input_padded, index_list, len_list = process_input(input_text)
#   pred_cwi = model_cwi.predict(input_padded)
#   pred_cwi_binary = np.argmax(pred_cwi, axis = 2)
#   complete_cwi_predictions = complete_missing_word(pred_cwi_binary, index_list, len_list)
#   bert_candidates =   get_bert_candidates(input_text, complete_cwi_predictions)
#   for word_to_replace, l_candidates in bert_candidates:
#     tuples_word_zipf = []
#     for w in l_candidates:
#       if w.isalpha():
#         tuples_word_zipf.append((w, zipf_frequency(w, 'en')))
#     tuples_word_zipf = sorted(tuples_word_zipf, key = lambda x: x[1], reverse=True)
#     new_text = re.sub(word_to_replace, tuples_word_zipf[0][0], new_text) 
#   print("Original text: ", input_text )
#   print("Simplified text:", new_text, "\n")

In [ ]:
# train_dataframe = pd.read_csv("News_Train.tsv",sep = '\t',header = None,names = ["ID","Sentence","complex_phrase_start_index","complex_phrase_end_index","Complex_phrase","num_native","num_n_native","native_complex","non-native complex","Binary values","probability values"])
# test_dataframe = pd.read_csv("News_Test.tsv",sep = '\t',header = None,names = ["ID","Sentence","complex_phrase_start_index","complex_phrase_end_index","Complex_phrase","num_native","num_n_native","native_complex","non-native complex","Binary values","probability values"])
# train_dataframe["Sentence"] = train_dataframe["Sentence"].apply(lambda x : x.lower())
# test_dataframe["Sentence"]= test_dataframe["Sentence"].apply(lambda x : x.lower())
# train_dataframe["Complex_phrase"] = train_dataframe["Complex_phrase"].apply(lambda x : x.lower())
# test_dataframe["Complex_phrase"] = test_dataframe["Complex_phrase"].apply(lambda x : x.lower())
# train_dataframe["Num_words"] = train_dataframe["Complex_phrase"].apply(lambda x : len(x.split()))
# test_dataframe["Num_words"] = test_dataframe["Complex_phrase"].apply(lambda x : len(x.split()))
# train_dataframe = train_dataframe[train_dataframe["Num_words"] == 1]
# test_dataframe = test_dataframe[test_dataframe["Num_words"] == 1]

In [ ]:
# import numpy as np
# import string
# from nltk.corpus import stopwords
# vocab_dict = {}
# sentences_list = np.array(train_dataframe["Sentence"])
# labels = np.array(train_dataframe["Binary values"])
# complex_words = np.array(train_dataframe["Complex_phrase"])
# complex_word_dict = {complex_words[i]: labels[i] for i in range(len(labels))}
# from nltk.tokenize import WordPunctTokenizer
# sen_tok_list = []
# wpt = WordPunctTokenizer()
# for sentence in sentences_list:
#   sen_tok = wpt.tokenize(sentence)
#   sen_tok_list.append(sen_tok)
# from nltk.stem.wordnet import WordNetLemmatizer
# wnlem = WordNetLemmatizer()
# for i in range(len(sen_tok_list)):
#   for j in range(len(sen_tok_list[i])):
#     sen_tok_list[i][j] = wnlem.lemmatize(sen_tok_list[i][j])
# final_word_list = []
# stop_words_list = stopwords.words('english')
# for i in range(len(sen_tok_list)):
#   temp_word_list = []
#   for j in range(len(sen_tok_list[i])):
#     if sen_tok_list[i][j] not in stop_words_list and sen_tok_list[i][j].isalnum() == True and sen_tok_list[i][j].isnumeric() == False:
#       temp_word_list.append(sen_tok_list[i][j])
#   final_word_list.append(temp_word_list)

# from gensim.models import Word2Vec
# word2vec_obj = Word2Vec(final_word_list, min_count=1,size= 40,workers =2,batch_words = 2000)
# vocab = word2vec_obj.wv.vocab.keys()

# word_list = []
# label_list = []
# seen_dict = {}
# for sentence in final_word_list:
#   for word in sentence:
#     if word not in seen_dict:
#       seen_dict[word] = 1
#       word_list.append(word)
#       if word not in complex_word_dict:
#         label_list.append(0)
#       else:
#         label_list.append(complex_word_dict[word])
# train_data_x = [] 
# max_length = 0
# for i in range(len(word_list)):
#     train_data_x.append(word2vec_obj.wv[word_list[i]])

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import BatchNormalization,Dense

# model = Sequential()
# model.add(Dense(25,activation=tf.nn.relu,kernel_initializer=tf.keras.initializers.he_normal(seed=42)))
# model.add(tf.keras.layers.BatchNormalization()) 
# model.add(Dense(1,activation=tf.nn.relu,kernel_initializer=tf.keras.initializers.he_normal(seed=64)))
# adam_opt = tf.keras.optimizers.Adam(learning_rate=0.001)
# model.compile(loss='binary_crossentropy', optimizer=adam_opt, metrics=['accuracy'])
# model.fit(train_data_x, label_list, epochs=50)